In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your Lang API key: ")

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

We need a persistence layer to store previously exchanged messages.

Wrapping our chat model in a minimal LangGraph application allows us to automatically persist the message history, simplifying the development of multi-turn applications.

LangGraph comes with a simple in-memory checkpointer, which we use below.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# this is a new graph
workflow = StateGraph(state_schema=MessagesState)

# this calls the model
def call_model(state:MessagesState):
    response = model.invoke(state["messages"])
    return { "messages": response }

# the graph has one node (besides the start)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


# This enables us to support multiple conversation threads with a single application,
# a common requirement when your application has multiple users.
config = {
    "configurable": {
        # each user (conversation) has a unique thread_id
        "thread_id": "test"
    }
}

In [9]:
from langchain_core.messages import HumanMessage

query = input("Enter your query: ")

input_messages = [HumanMessage(query)]
output = app.invoke({ "messages": input_messages }, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Va bene, se hai altre domande o se c'è qualcos'altro di cui vorresti parlare, sentiti libero di chiedere.


Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions